<a href="https://colab.research.google.com/github/moosunny/Transformer_Translator/blob/main/Seq2Seq_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

# 한국어-영어 대화체 데이터 다운로드(link:)
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Transformer_Translation/2_대화체.xlsx')
data

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [3]:
data_kor = data["원문"]
data_eng = data["번역문"]

data_kor_list = list(data_kor)
data_eng_list = list(data_eng)

# source 데이터 분리
train_src = data_kor_list[:89999]
val_src = data_kor_list[90000:94999]
test_src = data_kor_list[9500:]

# target 데이터 분리
train_target = data_eng_list[:89999]
val_target = data_eng_list[90000:94999]
test_target = data_eng_list[95000:]

def make_kor_eng_pair(src, target):
  return [{"kor": k, "eng": e} for k, e in zip(src, target)]

train_data = make_kor_eng_pair(train_src, train_target)
val_data = make_kor_eng_pair(val_src, val_target)
test_data = make_kor_eng_pair(test_src, test_target)
train_data[0]

{'kor': '이번 신제품 출시에 대한 시장의 반응은 어떤가요?',
 'eng': "How is the market's reaction to the newly released product?"}

In [4]:
train_data[0]["kor"]

'이번 신제품 출시에 대한 시장의 반응은 어떤가요?'

In [5]:
from transformers import AutoTokenizer
# from tokenizers import Tokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

# 한국어 토크나이저 활용(upstage)
tokenizer = AutoTokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")
# tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")
vocab_size = len(tokenizer.get_vocab().items()) # tokenizer vocab_size

# 패딩을 오른쪽으로 설정
tokenizer.padding_side = "right"
# pad 토큰 추가
tokenizer.pad_token = "<|pad|>"
# pad token index 설정
tokenizer.pad_token_id = 0  # 패딩 토큰의 인덱스 설정ㅁ


def make_token_ids(data):
  for item in tqdm(data):
    item["kor_token"] = tokenizer.tokenize(item["kor"], add_special_tokens= True)
    item["eng_token"] = tokenizer.tokenize(item["eng"], add_special_tokens= True)
    item["kor_ids"] = tokenizer.encode(item["kor"], padding= "max_length", max_length=50, truncation= True, return_tensors="pt", add_special_tokens= True).squeeze(0)
    item["eng_ids"] = tokenizer.encode(item["eng"], padding= "max_length", max_length=50, truncation= True, return_tensors="pt", add_special_tokens= True).squeeze(0)
  return data

train_data = make_token_ids(train_data)
val_data = make_token_ids(val_data)
test_data = make_token_ids(test_data)
train_data[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  0%|          | 0/89999 [00:00<?, ?it/s]

  0%|          | 0/4999 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

{'kor': '이번 신제품 출시에 대한 시장의 반응은 어떤가요?',
 'eng': "How is the market's reaction to the newly released product?",
 'kor_token': ['<|startoftext|>',
  '▁이번',
  '▁신제품',
  '▁출',
  '시에',
  '▁대한',
  '▁시장의',
  '▁반응',
  '은',
  '▁어떤',
  '가요',
  '?'],
 'eng_token': ['<|startoftext|>',
  '▁How',
  '▁is',
  '▁the',
  '▁market',
  "'",
  's',
  '▁reaction',
  '▁to',
  '▁the',
  '▁newly',
  '▁released',
  '▁product',
  '?'],
 'kor_ids': tensor([    1, 32646, 46967, 32337, 33981, 32268, 40755, 36246, 29538, 33478,
         40180, 28804,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'eng_ids': tensor([    1,  1602,   349,   272,  2668, 28742, 28713, 10285,   298,   272,
         12486,  5242,  2093, 28804,     0,     0,     0,     0,     0,     0,
           

In [6]:
print("eos 토큰 인덱스:",tokenizer.eos_token_id)
print("pad 토큰 인덱스:",tokenizer.pad_token_id)
print("sos 토큰 인덱스:",tokenizer.bos_token_id)
print("unk 토큰 인덱스:",tokenizer.unk_token_id)

eos 토큰 인덱스: 2
pad 토큰 인덱스: 0
sos 토큰 인덱스: 1
unk 토큰 인덱스: 0


In [7]:
import re

# 한글이 포함된 토큰만 필터링
kor_vocab = {token: idx for token, idx in tokenizer.get_vocab().items() if re.search(r'[ㄱ-ㅎㅏ-ㅣ가-힣]', token)}

# 영어 토큰만 필터링 (알파벳 포함 여부 확인)
eng_vocab = {token: idx for token, idx in tokenizer.get_vocab().items() if re.search(r'[a-zA-Z]', token)}

print("한국어 토큰 개수:",len(kor_vocab))
print("영어 토큰 개수:",len(eng_vocab))

한국어 토큰 개수: 12191
영어 토큰 개수: 25942


In [8]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        # 이미 tensor라면 그대로 사용
        batch_kor_ids = [example["kor_ids"].clone().detach() if isinstance(example["kor_ids"], torch.Tensor) else torch.tensor(example["kor_ids"], dtype=torch.long) for example in batch]
        batch_eng_ids = [example["eng_ids"].clone().detach() if isinstance(example["eng_ids"], torch.Tensor) else torch.tensor(example["eng_ids"], dtype=torch.long) for example in batch]

        # ✅ 리스트 → 하나의 `tensor(batch_size, seq_len)` 변환
        batch_kor_ids = torch.stack(batch_kor_ids)
        batch_eng_ids = torch.stack(batch_eng_ids)

        return {
            "kor_ids": batch_kor_ids,  # ✅ (batch_size, seq_len)
            "eng_ids": batch_eng_ids   # ✅ (batch_size, seq_len)
        }

    return collate_fn

batch_size = 128
collate_fn = get_collate_fn(pad_index=0)

def get_data_loader(dataset, batch_size, shuffle = False):
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
  return data_loader

train_loader = get_data_loader(train_data, batch_size=128, shuffle=False)
val_loader = get_data_loader(val_data, batch_size=128, shuffle=False)
test_loader = get_data_loader(test_data, batch_size=128, shuffle=False)

In [9]:
type(train_data[0]["kor_ids"])

torch.Tensor

In [10]:
for batch in test_loader:
  print(batch["kor_ids"].shape)
  break

torch.Size([128, 50])


In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

max_len = 50

# seed 고정
def seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed(42)

# class CustomDataset(Dataset):
#     def __init__(self, text, tokenizer, vocab_size, max_length = max_len, embedding_dim=1000):
#         self.text = text
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.vocab_size = vocab_size
#         self.embedding_dim = embedding_dim
#         self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

#     def __len__(self):
#         return len(self.text)

#     def __getitem__(self, idx):
#       text = self.text[idx]

#       encoding = self.tokenizer(
#                                 text,
#                                 padding="max_length",
#                                 max_length=self.max_length,
#                                 truncation= True,
#                                 return_tensors="pt"
#                                 )
#       input_idx = encoding["input_ids"]
#       embedded_inputs = self.emb(input_idx)


#       return embedded_inputs.squeeze(0) # (1, max_length, emb_dim)




# # 패딩을 오른쪽으로 설정
# tokenizer.padding_side = "right"

# # pad token index 설정
# tokenizer.pad_token_id = 0  # 패딩 토큰의 인덱스 설정


# # source 데이터 분리
# train_src = data_kor_list[:89999]
# val_src = data_kor_list[90000:94999]
# test_src = data_kor_list[9500:]

# # target 데이터 분리
# train_target = data_eng_list[:89999]
# val_target = data_eng_list[90000:94999]
# test_target = data_eng_list[95000:]


# # 데이터셋 생성
# train_src_dataset = CustomDataset(text = train_src, tokenizer = tokenizer,  vocab_size = vocab_size)
# val_src_dataset = CustomDataset(text = val_src, tokenizer = tokenizer,  vocab_size = vocab_size)
# test_src_dataset = CustomDataset(text = test_src, tokenizer = tokenizer,  vocab_size = vocab_size)

# train_input_data = train_data


# # train dataloader
# train_src_dataloader = DataLoader(train_src_dataset, batch_size=128, shuffle=False)
# val_src_dataloader = DataLoader(val_src_dataset, batch_size=128, shuffle=False)
# test_src_dataloader = DataLoader(test_src_dataset, batch_size=128, shuffle=False)

In [12]:
# target data
# train_target_dataset = CustomDataset(text = train_target, tokenizer = tokenizer,  vocab_size = vocab_size)
# val_target_dataset = CustomDataset(text = val_target, tokenizer = tokenizer,  vocab_size = vocab_size)
# test_target_dataset = CustomDataset(text = test_target, tokenizer = tokenizer,  vocab_size = vocab_size)

# train_target_dataloader = DataLoader(train_target_dataset, batch_size=128, shuffle=False)
# val_target_dataloader = DataLoader(val_target_dataset, batch_size=128, shuffle=False)
# test_target_dataloader = DataLoader(test_target_dataset, batch_size=128, shuffle=False)

In [13]:
import random

class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [38]:
input_dim = len(tokenizer.get_vocab().items())
output_dim = len(tokenizer.get_vocab().items())
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

In [39]:
print(input_dim,"input_dim")
print(output_dim,"output_dim")

64000 input_dim
64000 output_dim


In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,426,710 trainable parameters


In [41]:
pad_index = tokenizer.pad_token_id
print(pad_index)

0


In [43]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index= pad_index)

def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        print(batch["kor_ids"].dtype)  # float32/int64/long 확인
        print(batch["kor_ids"].min(), batch["kor_ids"].max())  # 범위 체크
        print(batch["eng_ids"].min(), batch["eng_ids"].max())  # 범위 체크
        print(batch["kor_ids"].device)
        src = batch["kor_ids"].to(device).long()
        print(src.dtype)
        trg = batch["eng_ids"].to(device).long()
        print(src.shape)
        print(trg.shape)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        print(output.shape)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [44]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["kor_ids"].to(device).long()
            trg = batch["eng_ids"].to(device).long()
            print(src.shape)
            print(trg.shape)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            print(output.shape)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [45]:
from tqdm.notebook import tqdm

n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        val_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]

torch.int64
tensor(0) tensor(62733)
tensor(0) tensor(28808)
cpu


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
